목적 : listed data Modeling from standard scaled 
===================

import modules

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

read csv files

In [8]:
train = pd.read_csv('../datasets/Train_Test/listed_smoted_train_data.csv')
test = pd.read_csv('../datasets/Train_Test/listed_test_data.csv')

In [9]:
# train_life_cycle = train[['도입기', '성장기', '성숙기', '쇠퇴기']]
# test_life_cycle = test[['도입기', '성장기', '성숙기', '쇠퇴기']]

selected features

In [10]:
x_train = train[['CASH FLOW 대 부채비율', '당좌비율', '순운전자본비율', '자기자본구성비율','경영자본순이익률',
                 '총자본영업이익률', '매출액영업이익률', '금융비용부담률', '이윤분배율', '유형자산회전율',
                 '상장년수', 'PCR', '쭈피처', '도입기', '성장기', '성숙기', '쇠퇴기']]

x_test = test[['CASH FLOW 대 부채비율', '당좌비율', '순운전자본비율', '자기자본구성비율','경영자본순이익률',
                 '총자본영업이익률', '매출액영업이익률', '금융비용부담률', '이윤분배율', '유형자산회전율',
                 '상장년수', 'PCR', '쭈피처', '도입기', '성장기', '성숙기', '쇠퇴기']]

y_train = train['부실판단']
y_test = test['부실판단']

# describe

In [11]:
x_train[train['부실판단'] == 0].describe()

,CASH FLOW 대 부채비율,당좌비율,순운전자본비율,자기자본구성비율,경영자본순이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,이윤분배율,유형자산회전율,상장년수,PCR,쭈피처,도입기,성장기,성숙기,쇠퇴기
count,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,4.050000e+02,405.000000,405.000000,405.000000,405.000000,405.000000
mean,18.392711,259.986972,27.406526,64.650300,2.651563,3.232374,3.513530,0.617278,-16.953764,4.500965,13.027461,-3.617906e+05,0.259556,0.306173,0.197531,0.296296,0.200000
std,39.486382,292.826805,21.564893,17.419138,11.804719,8.029468,14.201824,2.246628,133.721570,7.251934,7.759981,3.196792e+06,0.204819,0.461472,0.398629,0.457188,0.400495
min,-30.840000,19.750000,-42.750000,19.790000,-58.955600,-21.744800,-56.410800,-11.230100,-1507.750000,0.000000,4.000000,-2.892257e+07,0.020767,0.000000,0.000000,0.000000,0.000000
25%,-3.000000,105.000000,13.000000,52.889387,-1.384838,-0.452442,-1.491769,-0.360000,-4.578693,2.000000,7.000000,-2.665364e+04,0.124666,0.000000,0.000000,0.000000,0.000000
50%,6.372235,155.000000,26.433084,66.128257,3.817866,3.070000,4.820000,0.420000,16.079327,2.545456,10.000000,1.314796e+03,0.191642,0.000000,0.000000,0.000000,0.000000
75%,22.760000,280.000000,42.000000,78.958331,8.100000,6.783909,10.290000,1.472505,33.000000,4.296274,18.993755,1.301386e+04,0.325581,1.000000,0.000000,1.000000,0.000000
max,205.680000,2092.450000,84.240000,95.810000,48.247500,48.710600,46.326400,10.308400,130.250000,80.160000,36.620000,2.330352e+07,1.584983,1.000000,1.000000,1.000000,1.000000


In [12]:
x_train[train['부실판단'] == 1].describe()

,CASH FLOW 대 부채비율,당좌비율,순운전자본비율,자기자본구성비율,경영자본순이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,이윤분배율,유형자산회전율,상장년수,PCR,쭈피처,도입기,성장기,성숙기,쇠퇴기
count,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,4.050000e+02,405.000000,405.000000,405.000000,405.000000,405.000000
mean,-5.146227,171.545111,9.340687,50.623951,-12.745296,-5.723411,-15.222683,2.524695,-79.503603,9.163692,17.815897,-8.315832e+04,0.403163,0.306173,0.197531,0.296296,0.200000
std,34.500272,267.844093,26.016506,19.399789,21.926196,11.221443,46.492986,3.647689,198.192817,24.370933,8.616713,3.475124e+06,0.651980,0.461472,0.398629,0.457188,0.400495
min,-207.540000,8.000000,-63.650000,1.610000,-128.958000,-65.968000,-581.323000,-7.806000,-1551.600000,0.000000,2.460000,-3.629954e+07,0.041921,0.000000,0.000000,0.000000,0.000000
25%,-11.000000,59.000000,-9.000000,37.024273,-18.930000,-8.194287,-18.150000,0.566740,-84.000000,1.528030,10.008596,-2.056238e+04,0.141931,0.000000,0.000000,0.000000,0.000000
50%,-1.000000,93.000000,9.892140,50.000000,-7.980000,-3.244014,-4.840000,1.849054,-13.304920,3.000000,17.278270,1.826813e+03,0.235280,0.000000,0.000000,0.000000,0.000000
75%,7.000000,152.000000,23.000000,64.000000,-0.500000,0.370000,0.820000,3.520000,0.000000,6.444103,23.000000,2.323910e+04,0.436624,1.000000,0.000000,1.000000,0.000000
max,109.000000,2258.940000,84.930000,95.000000,77.924000,13.207500,26.026400,23.888000,66.800000,264.000000,48.200000,1.307304e+07,6.913460,1.000000,1.000000,1.000000,1.000000


# standard scaling

In [13]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

labels = x_train.columns

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = pd.DataFrame(data=x_train, columns = labels)
x_test = pd.DataFrame(data=x_test, columns = labels)

# modeling

In [14]:
default = pd.DataFrame()
best = pd.DataFrame()
validation = pd.DataFrame()

## LogisticRegression

### 기본 모델

In [15]:
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)


print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 스코어: {f1:.2f}')
print(f'ROC AUC 스코어: {roc_auc:.2f}')

# 결과 저장
result_logit = {
    'Accuracy' : round(accuracy, 3),
    'Precision' : round(precision, 3),
    'Recall' : round(recall, 3),
    'F1' : round(f1, 3),
    'ROC AUC' : round(roc_auc, 3)
}
default['LogisticRegression'] = result_logit
default

Accuracy: 0.772
Precision: 0.792
Recall: 0.752
F1 스코어: 0.77
ROC AUC 스코어: 0.77


,LogisticRegression
Accuracy,0.772
Precision,0.792
Recall,0.752
F1,0.772
ROC AUC,0.772


### 하이퍼 파라미터 조정

In [16]:
# 하이퍼파라미터 범위 지정
param_grid = {
    'C': np.arange(0, 10.0, 0.1),  # 규제 강도
    'penalty': ['l1', 'l2', 'elasticnet'],  # 규제 유형
    'solver': ['liblinear', 'saga']  # 최적화 알고리즘
}

model = LogisticRegression()
random_logit = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_logit.fit(x_train, y_train)

best_model = random_logit.best_estimator_

y_pred = best_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Logistic 하이퍼파라미터 조정 평가 지표")
print("Best Parameters:", random_logit.best_params_)
print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'ROC AUC Score: {roc_auc:.3f}')

# 파라미터 조정 결과 저장
result0_logit = {
    'Accuracy' : round(accuracy, 3),
    'Precision' : round(precision, 3),
    'Recall' : round(recall, 3),
    'F1' : round(f1, 3),
    'ROC AUC' : round(roc_auc, 3),
    'Best Parameters' : random_logit.best_params_
}
best['LogisticRegression'] = result0_logit
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Logistic 하이퍼파라미터 조정 평가 지표
Best Parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 1.8}
Accuracy: 0.772
Precision: 0.792
Recall: 0.752
F1 Score: 0.772
ROC AUC Score: 0.772


,LogisticRegression
Accuracy,0.772
Precision,0.792
Recall,0.752
F1,0.772
ROC AUC,0.772
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ..."


### 최적 파라미터 검증

In [17]:
# 모델 생성
cv_logit_model = LogisticRegression(
    solver=random_logit.best_params_['solver'],
    penalty=random_logit.best_params_['penalty'],
    C=random_logit.best_params_['C']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_logit_model, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_logit = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['LogisticRegression'] = cv_result_logit
validation

CV_Accuracy_Scores: [0.7037037  0.7654321  0.81481481 0.78395062 0.87654321]
CV_Precision_Scores: [0.72       0.7311828  0.82278481 0.77380952 0.8961039 ]
CV_Recall_Scores: [0.66666667 0.83950617 0.80246914 0.80246914 0.85185185]
CV_F1_Scores: [0.69230769 0.7816092  0.8125     0.78787879 0.87341772]
CV_ROC/AUC: [0.79530559 0.82319768 0.87959153 0.90245389 0.9276025 ]

=======교차검증 결과=======
CV_Accuracy_mean: 0.789
CV_Precision_mean: 0.789
CV_Recall_mean: 0.793
CV_F1_스코어_mean: 0.790
CV_ROC_AUC+스코어_mean: 0.866


,LogisticRegression
Accuracy,0.789
Precision,0.789
Recall,0.793
F1,0.790
ROC AUC,0.866


## RandomForestClassifier

### 기본 모델

In [18]:
# Random Forest 모델 생성 및 학습
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_rf = rf_model.predict(x_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)

print("Random Forest 모델 평가 지표")
print(f'Accuracy: {accuracy_rf:.3f}')
print(f'Precision: {precision_rf:.3f}')
print(f'Recall: {recall_rf:.3f}')
print(f'F1 스코어: {f1_rf:.2f}')
print(f'ROC AUC 스코어: {roc_auc_rf:.2f}')

# 결과 저장
result_rf = {
    'Accuracy' : round(accuracy_rf, 3),
    'Precision' : round(precision_rf, 3),
    'Recall' : round(recall_rf, 3),
    'F1' : round(f1_rf, 3),
    'ROC AUC' : round(roc_auc_rf, 3)
}
default['RandomForestClassifier'] = result_rf
default

Random Forest 모델 평가 지표
Accuracy: 0.822
Precision: 0.817
Recall: 0.842
F1 스코어: 0.83
ROC AUC 스코어: 0.82


,LogisticRegression,RandomForestClassifier
Accuracy,0.772,0.822
Precision,0.792,0.817
Recall,0.752,0.842
F1,0.772,0.829
ROC AUC,0.772,0.822


### 하이퍼 파라미터 조정

In [23]:
random_search = {'n_estimators': np.arange(10, 100, 1),
                 'max_depth': np.arange(0, 9, 1),
                 'min_samples_split': np.arange(1, 9, 1),
                 'min_samples_leaf': np.arange(1, 9, 1)}

clf = RandomForestClassifier()
random_rf = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 10, 
                               cv = 5, verbose= 1, random_state= 101, n_jobs = -1)
random_rf.fit(x_train,y_train)
random_pf = random_rf.best_estimator_.predict(x_test)

accuracy_rdrf = accuracy_score(y_test, random_pf)
precision_rdrf = precision_score(y_test, random_pf)
recall_rdrf = recall_score(y_test, random_pf)
f1_rdrf = f1_score(y_test, random_pf)
roc_auc_rdrf = roc_auc_score(y_test, random_pf)

print("Random Forest 모델 하이퍼파라미터 조정 평가 지표")
print(random_rf.best_params_)
print(f'Accuracy: {accuracy_rdrf:.3f}')
print(f'Precision: {precision_rdrf:.3f}')
print(f'Recall: {recall_rdrf:.3f}')
print(f'F1 스코어: {f1_rdrf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rdrf:.3f}')

# 파라미터 조정 결과 저장
result0_rf = {
    'Accuracy' : round(accuracy_rdrf, 3),
    'Precision' : round(precision_rdrf, 3),
    'Recall' : round(recall_rdrf, 3),
    'F1' : round(f1_rdrf, 3),
    'ROC AUC' : round(roc_auc_rdrf, 3),
    'Best Parameters' : random_rf.best_params_
}
best['RandomForestClassifier'] = result0_rf
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Random Forest 모델 하이퍼파라미터 조정 평가 지표
{'n_estimators': 29, 'min_samples_split': 7, 'min_samples_leaf': 8, 'max_depth': 7}
Accuracy: 0.838
Precision: 0.835
Recall: 0.851
F1 스코어: 0.843
ROC AUC 스코어: 0.837


,LogisticRegression,RandomForestClassifier
Accuracy,0.772,0.838
Precision,0.792,0.835
Recall,0.752,0.851
F1,0.772,0.843
ROC AUC,0.772,0.837
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 29, 'min_samples_split': 7, '..."


### 최적 파라미터 검증

In [24]:
# 모델 생성
cv_model_rf = RandomForestClassifier(
    n_estimators=random_rf.best_params_['n_estimators'],
    min_samples_split=random_rf.best_params_['min_samples_split'],
    min_samples_leaf=random_rf.best_params_['min_samples_leaf'],
    max_depth=random_rf.best_params_['max_depth']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_rf = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['RandomForestClassifier'] = cv_result_rf
validation

CV_Accuracy_Scores: [0.7654321  0.75925926 0.84567901 0.87037037 0.84567901]
CV_Precision_Scores: [0.73195876 0.75268817 0.87671233 0.86046512 0.8372093 ]
CV_Recall_Scores: [0.88888889 0.85185185 0.79012346 0.91358025 0.91358025]
CV_F1_Scores: [0.78688525 0.77456647 0.82580645 0.88757396 0.85882353]
CV_ROC/AUC: [0.83859168 0.85550983 0.91053193 0.94909313 0.92729767]

=======교차검증 결과=======
CV_Accuracy_mean: 0.817
CV_Precision_mean: 0.812
CV_Recall_mean: 0.872
CV_F1_스코어_mean: 0.827
CV_ROC_AUC+스코어_mean: 0.896


,LogisticRegression,RandomForestClassifier
Accuracy,0.789,0.817
Precision,0.789,0.812
Recall,0.793,0.872
F1,0.790,0.827
ROC AUC,0.866,0.896


## AdaBoostClassifier

### 기본 모델

In [25]:
# AdaBoost 모델 생성 및 학습
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_adaboost = adaboost_model.predict(x_test)

accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
precision_adaboost = precision_score(y_test, y_pred_adaboost)
recall_adaboost = recall_score(y_test, y_pred_adaboost)
f1_adaboost = f1_score(y_test, y_pred_adaboost)
roc_auc_adaboost = roc_auc_score(y_test, y_pred_adaboost)

print("AdaBoost 모델 평가 지표:")
print(f'Accuracy: {accuracy_adaboost:.3f}')
print(f'Precision: {precision_adaboost:.3f}')
print(f'Recall: {recall_adaboost:.3f}')
print(f'F1 스코어: {f1_adaboost:.3f}')
print(f'ROC AUC 스코어: {roc_auc_adaboost:.3f}')

# 결과 저장
result_adaboost = {
    'Accuracy' : round(accuracy_adaboost, 3),
    'Precision' : round(precision_adaboost, 3),
    'Recall' : round(recall_adaboost, 3),
    'F1' : round(f1_adaboost, 3),
    'ROC AUC' : round(roc_auc_adaboost, 3)
}
default['AdaBoostClassifier'] = result_adaboost
default

AdaBoost 모델 평가 지표:
Accuracy: 0.772
Precision: 0.804
Recall: 0.733
F1 스코어: 0.767
ROC AUC 스코어: 0.773


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier
Accuracy,0.772,0.822,0.772
Precision,0.792,0.817,0.804
Recall,0.752,0.842,0.733
F1,0.772,0.829,0.767
ROC AUC,0.772,0.822,0.773


### 하이퍼 파라미터 조정

In [26]:
adaboost_model = AdaBoostClassifier()

param_grid = {
    'n_estimators': np.arange(10, 100, 1),  # 트리 개수
    'learning_rate': np.arange(0.001, 1, 0.001)  # 학습률
}

random_adaboost = RandomizedSearchCV(estimator=adaboost_model, param_distributions=param_grid, n_iter=10, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_adaboost.fit(x_train, y_train)

best_model = random_adaboost.best_estimator_

y_pred_adaboost = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
precision_adaboost = precision_score(y_test, y_pred_adaboost)
recall_adaboost = recall_score(y_test, y_pred_adaboost)
f1_adaboost = f1_score(y_test, y_pred_adaboost)
roc_auc_adaboost = roc_auc_score(y_test, y_pred_adaboost)

print("Best Parameters:", random_adaboost.best_params_)
print("AdaBoost 모델 하이퍼파라미터 조정 평가 지표:")
print(f'Accuracy: {accuracy_adaboost:.3f}')
print(f'Precision: {precision_adaboost:.3f}')
print(f'Recall: {recall_adaboost:.3f}')
print(f'F1 Score: {f1_adaboost:.3f}')
print(f'ROC AUC Score: {roc_auc_adaboost:.3f}')

# 파라미터 조정 결과 저장
result0_adaboost = {
    'Accuracy' : round(accuracy_adaboost, 3),
    'Precision' : round(precision_adaboost, 3),
    'Recall' : round(recall_adaboost, 3),
    'F1' : round(f1_adaboost, 3),
    'ROC AUC' : round(roc_auc_adaboost, 3),
    'Best Parameters' : random_adaboost.best_params_
}
best['AdaBoostClassifier'] = result0_adaboost
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Best Parameters: {'n_estimators': 34, 'learning_rate': 0.41400000000000003}
AdaBoost 모델 하이퍼파라미터 조정 평가 지표:
Accuracy: 0.817
Precision: 0.849
Recall: 0.782
F1 Score: 0.814
ROC AUC Score: 0.818


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier
Accuracy,0.772,0.838,0.817
Precision,0.792,0.835,0.849
Recall,0.752,0.851,0.782
F1,0.772,0.843,0.814
ROC AUC,0.772,0.837,0.818
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 29, 'min_samples_split': 7, '...","{'n_estimators': 34, 'learning_rate': 0.414000..."


### 최적 파라미터 검증

In [27]:
# 모델 생성 
# AdaBoostClassifier 최적파라미터 적용 Crossvalidation
cv_model_adaboost = AdaBoostClassifier(
    n_estimators=random_adaboost.best_params_['n_estimators'],
    learning_rate=random_adaboost.best_params_['learning_rate']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_adaboost = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['AdaBoostClassifier'] = cv_result_adaboost

CV_Accuracy_Scores: [0.74074074 0.72839506 0.82098765 0.88888889 0.83333333]
CV_Precision_Scores: [0.72413793 0.69473684 0.88235294 0.85393258 0.875     ]
CV_Recall_Scores: [0.77777778 0.81481481 0.74074074 0.9382716  0.77777778]
CV_F1_Scores: [0.75       0.75       0.80536913 0.89411765 0.82352941]
CV_ROC/AUC: [0.82098765 0.82639841 0.8861454  0.93613778 0.8783722 ]

=======교차검증 결과=======
CV_Accuracy_mean: 0.802
CV_Precision_mean: 0.806
CV_Recall_mean: 0.810
CV_F1_스코어_mean: 0.805
CV_ROC_AUC+스코어_mean: 0.870


## BaggingClassifier

### 기본 모델

In [28]:
# Bagging 모델 생성 및 학습
bagging_model = BaggingClassifier()
bagging_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_bagging = bagging_model.predict(x_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging)
recall_bagging = recall_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
roc_auc_bagging = roc_auc_score(y_test, y_pred_bagging)

print("Bagging 모델 평가 지표:")
print(f'Accuracy: {accuracy_bagging:.3f}')
print(f'Precision: {precision_bagging:.3f}')
print(f'Recall: {recall_bagging:.3f}')
print(f'F1 스코어: {f1_bagging:.3f}')
print(f'ROC AUC 스코어: {roc_auc_bagging:.3f}')

# 결과 저장
result_bagging = {
    'Accuracy' : round(accuracy_bagging, 3),
    'Precision' : round(precision_bagging, 3),
    'Recall' : round(recall_bagging, 3),
    'F1' : round(f1_bagging, 3),
    'ROC AUC' : round(roc_auc_bagging, 3)
}
default['BaggingClassifier'] = result_bagging
default

Bagging 모델 평가 지표:
Accuracy: 0.782
Precision: 0.837
Recall: 0.713
F1 스코어: 0.770
ROC AUC 스코어: 0.784


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
Accuracy,0.772,0.822,0.772,0.782
Precision,0.792,0.817,0.804,0.837
Recall,0.752,0.842,0.733,0.713
F1,0.772,0.829,0.767,0.770
ROC AUC,0.772,0.822,0.773,0.784


### 하이퍼 파라미터 조정

In [29]:
bagging_model = BaggingClassifier()

param_grid = {
    'n_estimators': np.arange(1, 100, 1),
    'max_samples': np.arange(0.1, 1.0, 0.1),
    'max_features': np.arange(0.1, 1, 0.1)
}

random_bagging = RandomizedSearchCV(estimator=bagging_model, param_distributions=param_grid, n_iter=10, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_bagging.fit(x_train, y_train)

# 최적의 모델 저장
best_model = random_bagging.best_estimator_

# 최적의 모델로 예측 수행
y_pred_bagging = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging)
recall_bagging = recall_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
roc_auc_bagging = roc_auc_score(y_test, y_pred_bagging)

print("Best Parameters:", random_bagging.best_params_)
print("Bagging 모델 하이퍼파라미터 평가 지표:")
print(f'Accuracy: {accuracy_bagging:.3f}')
print(f'Precision: {precision_bagging:.3f}')
print(f'Recall: {recall_bagging:.3f}')
print(f'F1 Score: {f1_bagging:.3f}')
print(f'ROC AUC Score: {roc_auc_bagging:.3f}')

# 파라미터 조정 결과 저장
result0_bagging = {
    'Accuracy' : round(accuracy_bagging, 3),
    'Precision' : round(precision_bagging, 3),
    'Recall' : round(recall_bagging, 3),
    'F1' : round(f1_bagging, 3),
    'ROC AUC' : round(roc_auc_bagging, 3),
    'Best Parameters' : random_bagging.best_params_
}
best['BaggingClassifier'] = result0_bagging
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Best Parameters: {'n_estimators': 79, 'max_samples': 0.8, 'max_features': 0.6}
Bagging 모델 하이퍼파라미터 평가 지표:
Accuracy: 0.832
Precision: 0.821
Recall: 0.861
F1 Score: 0.841
ROC AUC Score: 0.832


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
Accuracy,0.772,0.838,0.817,0.832
Precision,0.792,0.835,0.849,0.821
Recall,0.752,0.851,0.782,0.861
F1,0.772,0.843,0.814,0.841
ROC AUC,0.772,0.837,0.818,0.832
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 29, 'min_samples_split': 7, '...","{'n_estimators': 34, 'learning_rate': 0.414000...","{'n_estimators': 79, 'max_samples': 0.8, 'max_..."


### 최적 파라미터 검증

In [30]:
# 모델 생성 
# BaggingClassifier 최적파라미터 적용 Crossvalidation
cv_model_bagging = BaggingClassifier(
    n_estimators=random_bagging.best_params_['n_estimators'],
    max_samples=random_bagging.best_params_['max_samples'],
    max_features=random_bagging.best_params_['max_features']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_bagging = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['BaggingClassifier'] = cv_result_bagging

CV_Accuracy_Scores: [0.81481481 0.84567901 0.86419753 0.87654321 0.84567901]
CV_Precision_Scores: [0.77419355 0.8021978  0.85185185 0.85057471 0.84337349]
CV_Recall_Scores: [0.85185185 0.91358025 0.79012346 0.92592593 0.88888889]
CV_F1_Scores: [0.82758621 0.83529412 0.84810127 0.89820359 0.85882353]
CV_ROC/AUC: [0.90176802 0.88584057 0.92752629 0.95541838 0.95145557]

=======교차검증 결과=======
CV_Accuracy_mean: 0.849
CV_Precision_mean: 0.824
CV_Recall_mean: 0.874
CV_F1_스코어_mean: 0.854
CV_ROC_AUC+스코어_mean: 0.924


## SVC_linear

### 기본 모델

In [31]:
from sklearn.svm import SVC

# SVM 모델 생성 및 학습
svm_model = SVC(kernel='linear')
svm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_svm = svm_model.predict(x_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 스코어: {f1_svm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_svm:.3f}')

# 결과 저장
result_svc_linear = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3)
}
default['SVC_linear'] = result_svc_linear
default

SVM 모델 평가 지표:
Accuracy: 0.766
Precision: 0.789
Recall: 0.743
F1 스코어: 0.765
ROC AUC 스코어: 0.767


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear
Accuracy,0.772,0.822,0.772,0.782,0.766
Precision,0.792,0.817,0.804,0.837,0.789
Recall,0.752,0.842,0.733,0.713,0.743
F1,0.772,0.829,0.767,0.770,0.765
ROC AUC,0.772,0.822,0.773,0.784,0.767


### 하이퍼 파라미터 조정

In [32]:
svm_model = SVC()

param_grid = {
    'C': np.arange(0.1, 100, 0.1),  # 규제 매개변수
    'gamma': np.arange(0.001, 1, 0.001),  # 커널 계수
    'kernel': ['linear']  # 커널 타입
}

random_svc_linear = RandomizedSearchCV(estimator=svm_model, param_distributions=param_grid, n_iter=10, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_svc_linear.fit(x_train, y_train)

print("Best Parameters:", random_svc_linear.best_params_)
best_model = random_svc_linear.best_estimator_

y_pred_svm = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

# 평가 메트릭 출력
print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 Score: {f1_svm:.3f}')
print(f'ROC AUC Score: {roc_auc_svm:.3f}')

# 파라미터 조정 결과 저장
result0_svc_linear = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3),
    'Best Parameters' : random_svc_linear.best_params_
}
best['SVC_linear'] = result0_svc_linear
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Best Parameters: {'kernel': 'linear', 'gamma': 0.8270000000000001, 'C': 67.2}
SVM 모델 평가 지표:
Accuracy: 0.772
Precision: 0.792
Recall: 0.752
F1 Score: 0.772
ROC AUC Score: 0.772


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear
Accuracy,0.772,0.838,0.817,0.832,0.772
Precision,0.792,0.835,0.849,0.821,0.792
Recall,0.752,0.851,0.782,0.861,0.752
F1,0.772,0.843,0.814,0.841,0.772
ROC AUC,0.772,0.837,0.818,0.832,0.772
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 29, 'min_samples_split': 7, '...","{'n_estimators': 34, 'learning_rate': 0.414000...","{'n_estimators': 79, 'max_samples': 0.8, 'max_...","{'kernel': 'linear', 'gamma': 0.82700000000000..."


### 최적 파라미터 검증

In [33]:
# 모델 생성 
# SVC_linear 최적파라미터 적용 Crossvalidation
cv_model_scv_linear = SVC(
    kernel='linear',
    gamma=random_svc_linear.best_params_['gamma'],
    C=random_svc_linear.best_params_['C']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_linear = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['SVC_linear'] = cv_result_linear

CV_Accuracy_Scores: [0.71604938 0.75308642 0.80246914 0.7962963  0.87654321]
CV_Precision_Scores: [0.73972603 0.73563218 0.79518072 0.77272727 0.94202899]
CV_Recall_Scores: [0.66666667 0.79012346 0.81481481 0.83950617 0.80246914]
CV_F1_Scores: [0.7012987  0.76190476 0.80487805 0.80473373 0.86666667]
CV_ROC/AUC: [0.79408627 0.82441701 0.87989636 0.90367322 0.92653559]

=======교차검증 결과=======
CV_Accuracy_mean: 0.789
CV_Precision_mean: 0.797
CV_Recall_mean: 0.783
CV_F1_스코어_mean: 0.788
CV_ROC_AUC+스코어_mean: 0.866


## SVC_rbf

### 기본모델

In [34]:
# SVM 모델 생성 및 학습
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_svm = svm_model.predict(x_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 스코어: {f1_svm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_svm:.3f}')

# 결과 저장
result_svc_rbf = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3)
}
default['SVC_rbf'] = result_svc_rbf
default

SVM 모델 평가 지표:
Accuracy: 0.802
Precision: 0.810
Recall: 0.802
F1 스코어: 0.806
ROC AUC 스코어: 0.802


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf
Accuracy,0.772,0.822,0.772,0.782,0.766,0.802
Precision,0.792,0.817,0.804,0.837,0.789,0.810
Recall,0.752,0.842,0.733,0.713,0.743,0.802
F1,0.772,0.829,0.767,0.770,0.765,0.806
ROC AUC,0.772,0.822,0.773,0.784,0.767,0.802


### 하이퍼 파라미터 조정

In [35]:
svm_model = SVC()

param_grid = {
    'C': np.arange(0.01, 1, 0.01),  # Reduce the maximum value
    'gamma': np.arange(0.01, 0.1, 0.01),  # Reduce the maximum value
    'kernel': ['rbf']
}

random_svc_rbf = RandomizedSearchCV(estimator=svm_model, param_distributions=param_grid, n_iter=10, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_svc_rbf.fit(x_train, y_train)

print("Best Parameters:", random_svc_rbf.best_params_)
best_model = random_svc_rbf.best_estimator_

y_pred_svm = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

# 평가 메트릭 출력
print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 Score: {f1_svm:.3f}')
print(f'ROC AUC Score: {roc_auc_svm:.3f}')

# 파라미터 조정 결과 저장
result0_svc_rbf = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3),
    'Best Parameters' : random_svc_rbf.best_params_
}
best['SVC_rbf'] = result0_svc_rbf
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Best Parameters: {'kernel': 'rbf', 'gamma': 0.08, 'C': 0.79}
SVM 모델 평가 지표:
Accuracy: 0.812
Precision: 0.814
Recall: 0.822
F1 Score: 0.818
ROC AUC Score: 0.812


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf
Accuracy,0.772,0.838,0.817,0.832,0.772,0.812
Precision,0.792,0.835,0.849,0.821,0.792,0.814
Recall,0.752,0.851,0.782,0.861,0.752,0.822
F1,0.772,0.843,0.814,0.841,0.772,0.818
ROC AUC,0.772,0.837,0.818,0.832,0.772,0.812
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 29, 'min_samples_split': 7, '...","{'n_estimators': 34, 'learning_rate': 0.414000...","{'n_estimators': 79, 'max_samples': 0.8, 'max_...","{'kernel': 'linear', 'gamma': 0.82700000000000...","{'kernel': 'rbf', 'gamma': 0.08, 'C': 0.79}"


### 최적 파라미터 검증

In [36]:
# 모델 생성 
# SVC_linear 최적파라미터 적용 Crossvalidation
cv_model_scv_rbf = SVC(
    kernel='rbf',
    gamma=random_svc_rbf.best_params_['gamma'],
    C=random_svc_rbf.best_params_['C']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_rbf = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['SVC_rbf'] = cv_result_rbf

CV_Accuracy_Scores: [0.7654321  0.77777778 0.80864198 0.80864198 0.89506173]
CV_Precision_Scores: [0.79452055 0.73684211 0.82051282 0.78409091 0.8902439 ]
CV_Recall_Scores: [0.71604938 0.86419753 0.79012346 0.85185185 0.90123457]
CV_F1_Scores: [0.75324675 0.79545455 0.80503145 0.81656805 0.89570552]
CV_ROC/AUC: [0.83721994 0.83904893 0.89894833 0.91708581 0.95153178]

=======교차검증 결과=======
CV_Accuracy_mean: 0.811
CV_Precision_mean: 0.805
CV_Recall_mean: 0.825
CV_F1_스코어_mean: 0.813
CV_ROC_AUC+스코어_mean: 0.889


## LGBMClassifier

In [37]:
# LGBM 모델 생성 및 학습
lgbm_model = LGBMClassifier()
lgbm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_lgbm = lgbm_model.predict(x_test)

accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm)
precision_lgbm = precision_score(y_test, y_pred_lgbm)
recall_lgbm = recall_score(y_test, y_pred_lgbm)
f1_lgbm = f1_score(y_test, y_pred_lgbm)
roc_auc_lgbm = roc_auc_score(y_test, y_pred_lgbm)

print("LightGBM 모델 평가 지표:")
print(f'Accuracy: {accuracy_lgbm:.3f}')
print(f'Precision: {precision_lgbm:.3f}')
print(f'Recall: {recall_lgbm:.3f}')
print(f'F1 스코어: {f1_lgbm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_lgbm:.3f}')

# 결과 저장
result_lgbm = {
    'Accuracy' : round(accuracy_lgbm, 3),
    'Precision' : round(precision_lgbm, 3),
    'Recall' : round(recall_lgbm, 3),
    'F1' : round(f1_lgbm, 3),
    'ROC AUC' : round(roc_auc_lgbm, 3)
}
default['LGBMClassifier'] = result_lgbm
default

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 405, number of negative: 405
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001144 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3159
[LightGBM] [Info] Number of data points in the train set: 810, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier
Accuracy,0.772,0.822,0.772,0.782,0.766,0.802,0.822
Precision,0.792,0.817,0.804,0.837,0.789,0.810,0.844
Recall,0.752,0.842,0.733,0.713,0.743,0.802,0.802
F1,0.772,0.829,0.767,0.770,0.765,0.806,0.822
ROC AUC,0.772,0.822,0.773,0.784,0.767,0.802,0.823


### 파라미터 조정

In [38]:
param_grid = {
        'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1],
        'n_estimators': np.arange(1, 100, 1),
        'max_depth': np.arange(1, 10, 1),
        'num_leaves': np.arange(1, 10, 1),
        'min_child_samples': np.arange(1, 10, 1)
    }

lgbm = LGBMClassifier()
random_lgbm = RandomizedSearchCV(lgbm, param_grid, cv = 5, n_jobs = -1, n_iter=10)
random_lgbm.fit(x_train,y_train)
random_pf = random_lgbm.best_estimator_.predict(x_test)

accuracy_rdrf = accuracy_score(y_test, random_pf)
precision_rdrf = precision_score(y_test, random_pf)
recall_rdrf = recall_score(y_test, random_pf)
f1_rdrf = f1_score(y_test, random_pf)
roc_auc_rdrf = roc_auc_score(y_test, random_pf)

print("LGBM 모델 하이퍼파라미터 조정 평가 지표")
print(random_lgbm.best_params_)
print(f'Accuracy: {accuracy_rdrf:.3f}')
print(f'Precision: {precision_rdrf:.3f}')
print(f'Recall: {recall_rdrf:.3f}')
print(f'F1 스코어: {f1_rdrf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rdrf:.3f}')

# 파라미터 조정 결과 저장
result0_lgbm = {
    'Accuracy' : round(accuracy_rdrf, 3),
    'Precision' : round(precision_rdrf, 3),
    'Recall' : round(recall_rdrf, 3),
    'F1' : round(f1_rdrf, 3),
    'ROC AUC' : round(roc_auc_rdrf, 3),
    'Best Parameters' : random_lgbm.best_params_
}
best['LGBMClassifier'] = result0_lgbm
best

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 405, number of negative: 405
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000617 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3159
[LightGBM] [Info] Number of data points in the train set: 810, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LGBM 모델 하이퍼파라미터 조정 평가 지표
{'num_leaves': 4, 'n_estimators': 87, 'min_child_samples': 2, 'max_depth': 6, 'learning_rate': 1}
Accuracy: 0.792
Precision: 0.812
Recall: 0.772
F1 스코어: 0.792
ROC AUC 스코어: 0.792


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier
Accuracy,0.772,0.838,0.817,0.832,0.772,0.812,0.792
Precision,0.792,0.835,0.849,0.821,0.792,0.814,0.812
Recall,0.752,0.851,0.782,0.861,0.752,0.822,0.772
F1,0.772,0.843,0.814,0.841,0.772,0.818,0.792
ROC AUC,0.772,0.837,0.818,0.832,0.772,0.812,0.792
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 29, 'min_samples_split': 7, '...","{'n_estimators': 34, 'learning_rate': 0.414000...","{'n_estimators': 79, 'max_samples': 0.8, 'max_...","{'kernel': 'linear', 'gamma': 0.82700000000000...","{'kernel': 'rbf', 'gamma': 0.08, 'C': 0.79}","{'num_leaves': 4, 'n_estimators': 87, 'min_chi..."


### 최적 파라미터 검증

In [39]:
# 모델 생성 
# lgbm 최적파라미터 적용 Crossvalidation
cv_model_lgbm = LGBMClassifier(
    num_leaves=random_lgbm.best_params_['num_leaves'],
    n_estimators=random_lgbm.best_params_['n_estimators'],
    min_child_samples=random_lgbm.best_params_['min_child_samples'],
    max_depth=random_lgbm.best_params_['max_depth'],
    learning_rate=random_lgbm.best_params_['learning_rate']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_lgbm = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['LGBMClassifier'] = cv_result_lgbm

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 324, number of negative: 324
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2569
[LightGBM] [Info] Number of data points in the train set: 648, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 324, number of negative: 324
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000309 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2574
[LightGBM] [Info] Number of data points in the train set: 648, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500

#### 딥러닝

In [145]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# import tensorflow as tf

In [146]:
# model = Sequential()
# model.add(Dense(256, input_dim = 12, activation = 'relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation = 'relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation = 'sigmoid'))

In [147]:
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# history = model.fit(x_train, y_train, epochs=200, batch_size=5)

# print("\n Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

In [148]:
# # 모델을 사용하여 테스트 데이터에 대한 예측 수행
# y_pred = model.predict(x_test)
# y_pred = binarize(y_pred, threshold=0.5)  # 예측값을 0.5 임계값을 기준으로 이진 분류로 변환

# # 정확도(accuracy) 계산
# accuracy = accuracy_score(y_test, y_pred)
# print(f"accuracy: {accuracy:.4f}")

# # F1 점수(f1 score) 계산
# f1 = f1_score(y_test, y_pred)
# print(f"F1-Score: {f1:.4f}")

# # 재현율(recall) 계산
# recall = recall_score(y_test, y_pred)
# print(f"recall: {recall:.4f}")

# # 정밀도(precision) 계산
# precision = precision_score(y_test, y_pred)
# print(f"precision: {precision:.4f}")

## XGBoost

기본 모델

In [50]:
# LGBM 모델 생성 및 학습
xgboost = XGBClassifier()
xgboost.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_xgboost = xgboost.predict(x_test)

accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
precision_xgboost = precision_score(y_test, y_pred_xgboost)
recall_xgboost = recall_score(y_test, y_pred_xgboost)
f1_xgboost = f1_score(y_test, y_pred_xgboost)
roc_auc_xgboost = roc_auc_score(y_test, y_pred_xgboost)

print("LightGBM 모델 평가 지표:")
print(f'Accuracy: {accuracy_xgboost:.3f}')
print(f'Precision: {precision_xgboost:.3f}')
print(f'Recall: {recall_xgboost:.3f}')
print(f'F1 스코어: {f1_xgboost:.3f}')
print(f'ROC AUC 스코어: {roc_auc_xgboost:.3f}')

# 결과 저장
result_xgboost = {
    'Accuracy' : round(accuracy_xgboost, 3),
    'Precision' : round(precision_xgboost, 3),
    'Recall' : round(recall_xgboost, 3),
    'F1' : round(f1_xgboost, 3),
    'ROC AUC' : round(roc_auc_xgboost, 3)
}
default['XGBClassifier'] = result_xgboost
default

LightGBM 모델 평가 지표:
Accuracy: 0.822
Precision: 0.830
Recall: 0.822
F1 스코어: 0.826
ROC AUC 스코어: 0.822


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier,XGBClassifier
Accuracy,0.772,0.822,0.772,0.782,0.766,0.802,0.822,0.822
Precision,0.792,0.817,0.804,0.837,0.789,0.810,0.844,0.830
Recall,0.752,0.842,0.733,0.713,0.743,0.802,0.802,0.822
F1,0.772,0.829,0.767,0.770,0.765,0.806,0.822,0.826
ROC AUC,0.772,0.822,0.773,0.784,0.767,0.802,0.823,0.822


### 하이퍼 파라미터 조정

In [54]:
# 파라미터 조정
xgboost_model = XGBClassifier()

param_grid = {
    'n_estimators': np.arange(1, 100, 1),  # 트리 개수
    'learning_rate': np.arange(0.01, 1, 0.01)  # 학습률
}

random_search = RandomizedSearchCV(estimator=xgboost_model, param_distributions=param_grid, n_iter=99, 
                                   scoring='accuracy', cv=7, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

best_model = random_search.best_estimator_

y_pred_xgboost = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
precision_xgboost = precision_score(y_test, y_pred_xgboost)
recall_xgboost = recall_score(y_test, y_pred_xgboost)
f1_xgboost = f1_score(y_test, y_pred_xgboost)
roc_auc_xgboost = roc_auc_score(y_test, y_pred_xgboost)

print("Best Parameters:", random_search.best_params_)
print("XGBoost 모델 하이퍼파라미터 조정 평가 지표:")
print(f'Accuracy: {accuracy_xgboost:.3f}')
print(f'Precision: {precision_xgboost:.3f}')
print(f'Recall: {recall_xgboost:.3f}')
print(f'F1 Score: {f1_xgboost:.3f}')
print(f'ROC AUC Score: {roc_auc_xgboost:.3f}')

result0_xgboost = {
    'Accuracy' : round(accuracy_xgboost, 3),
    'Precision' : round(precision_xgboost, 3),
    'Recall' : round(recall_xgboost, 3),
    'F1' : round(f1_xgboost, 3),
    'ROC AUC' : round(roc_auc_xgboost, 3),
    'Best Parameters' : random_search.best_params_
}
best['XGBClassifier'] = result0_xgboost
best

Fitting 7 folds for each of 99 candidates, totalling 693 fits
Best Parameters: {'n_estimators': 33, 'learning_rate': 0.31}
XGBoost 모델 하이퍼파라미터 조정 평가 지표:
Accuracy: 0.807
Precision: 0.825
Recall: 0.792
F1 Score: 0.808
ROC AUC Score: 0.807


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier,XGBClassifier
Accuracy,0.772,0.838,0.817,0.832,0.772,0.812,0.792,0.807
Precision,0.792,0.835,0.849,0.821,0.792,0.814,0.812,0.825
Recall,0.752,0.851,0.782,0.861,0.752,0.822,0.772,0.792
F1,0.772,0.843,0.814,0.841,0.772,0.818,0.792,0.808
ROC AUC,0.772,0.837,0.818,0.832,0.772,0.812,0.792,0.807
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 29, 'min_samples_split': 7, '...","{'n_estimators': 34, 'learning_rate': 0.414000...","{'n_estimators': 79, 'max_samples': 0.8, 'max_...","{'kernel': 'linear', 'gamma': 0.82700000000000...","{'kernel': 'rbf', 'gamma': 0.08, 'C': 0.79}","{'num_leaves': 4, 'n_estimators': 87, 'min_chi...","{'n_estimators': 33, 'learning_rate': 0.31}"


### 최적 파라미터 검증

In [55]:
# 최적 파라미터 검증
xgboost_model = XGBClassifier(
    n_estimators=random_search.best_params_['n_estimators'],
    learning_rate=random_search.best_params_['learning_rate']
)

# Cross Validation
cv_accuracy = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='roc_auc')

print('=======교차검증 결과=======')
print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 평균값=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_xgboost = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['XGBClassifier'] = cv_result_xgboost

=======교차검증 결과=======
CV_Accuracy_Scores: [0.80864198 0.86419753 0.83333333 0.88888889 0.84567901]
CV_Precision_Scores: [0.82051282 0.84705882 0.85526316 0.84615385 0.81111111]
CV_Recall_Scores: [0.79012346 0.88888889 0.80246914 0.95061728 0.90123457]
CV_F1_Scores: [0.80503145 0.86746988 0.82802548 0.89534884 0.85380117]
CV_ROC/AUC: [0.87943911 0.90809328 0.91495199 0.94619723 0.94345374]

=======교차검증 평균값=======
CV_Accuracy_mean: 0.848
CV_Precision_mean: 0.836
CV_Recall_mean: 0.867
CV_F1_스코어_mean: 0.850
CV_ROC_AUC_mean: 0.918


# 결과

In [51]:
validation

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier,XGBClassifier
Accuracy,0.789,0.817,0.802,0.849,0.789,0.811,0.835,0.835
Precision,0.789,0.812,0.806,0.824,0.797,0.805,0.830,0.821
Recall,0.793,0.872,0.810,0.874,0.783,0.825,0.842,0.854
F1,0.790,0.827,0.805,0.854,0.788,0.813,0.834,0.837
ROC AUC,0.866,0.896,0.870,0.924,0.866,0.889,0.907,0.912


In [52]:
default

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier,XGBClassifier
Accuracy,0.772,0.822,0.772,0.782,0.766,0.802,0.822,0.822
Precision,0.792,0.817,0.804,0.837,0.789,0.810,0.844,0.830
Recall,0.752,0.842,0.733,0.713,0.743,0.802,0.802,0.822
F1,0.772,0.829,0.767,0.770,0.765,0.806,0.822,0.826
ROC AUC,0.772,0.822,0.773,0.784,0.767,0.802,0.823,0.822


In [53]:
best

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier,XGBClassifier
Accuracy,0.772,0.838,0.817,0.832,0.772,0.812,0.792,0.827
Precision,0.792,0.835,0.849,0.821,0.792,0.814,0.812,0.845
Recall,0.752,0.851,0.782,0.861,0.752,0.822,0.772,0.812
F1,0.772,0.843,0.814,0.841,0.772,0.818,0.792,0.828
ROC AUC,0.772,0.837,0.818,0.832,0.772,0.812,0.792,0.828
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 29, 'min_samples_split': 7, '...","{'n_estimators': 34, 'learning_rate': 0.414000...","{'n_estimators': 79, 'max_samples': 0.8, 'max_...","{'kernel': 'linear', 'gamma': 0.82700000000000...","{'kernel': 'rbf', 'gamma': 0.08, 'C': 0.79}","{'num_leaves': 4, 'n_estimators': 87, 'min_chi...","{'n_estimators': 168, 'learning_rate': 0.803}"
